In [ ]:
# ⚙️ Setup
import subprocess, sys
try:
    import google.colab
    subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", "numba"])
except ImportError:
    pass

import numpy as np
from numba import cuda

print("⚠️  CUDA C++ is PRIMARY. Python/Numba for quick testing only.")
if cuda.is_available():
    print(f"GPU: {cuda.get_current_device().name}")

# Day 4: Graph Optimization

## 🎯 The Hook: Finding the Fastest Recipe

**A Michelin chef doesn't just follow recipes—they optimize them.** Should you prep all vegetables first, or interleave with cooking? Can two cooks work in parallel? Is the overhead of coordinating them worth it for a simple dish?

```
The Optimization Questions:
━━━━━━━━━━━━━━━━━━━━━━━━━━━
1. WHEN to use graphs?
   • 10 small kernels: YES! (50% overhead → 5%)
   • 1 large kernel:   NO!  (0.1% overhead → still 0.1%)

2. HOW to structure graphs?
   • Deep chain: K1→K2→K3→K4→K5 (serial, simple)
   • Wide fork:  K1→[K2,K3,K4]→K5 (parallel, faster!)

3. WHAT'S the breakeven point?
   • Setup cost: ~100μs (capture + instantiate)
   • Savings per launch: ~50μs (10 kernels × 5μs)
   • Breakeven: 100÷50 = 2 iterations!
   • After 100 iterations: 50x faster total!
```

**Today you'll learn:** How to analyze, optimize, and know when graphs help—turning good code into great code!

---

## 📚 Learning Objectives

By the end of this lesson, you will be able to:
1. **Analyze** when CUDA graphs provide benefit vs. add overhead
2. **Calculate breakeven points** for graph usage decisions
3. **Optimize graph structure** for maximum parallelism
4. **Profile** graph execution to identify bottlenecks
5. **Apply best practices** for production graph usage

---

## 🃏 Concept Card: Finding the Fastest Recipe

```
┌─────────────────────────────────────────────────────────────────┐
│  GRAPH OPTIMIZATION = CHEF'S EFFICIENCY ANALYSIS               │
├─────────────────────────────────────────────────────────────────┤
│                                                                 │
│  DECISION MATRIX - When to Use Graphs:                          │
│  ┌─────────────────────────────────────────────────────────┐   │
│  │                                                         │   │
│  │  ✅ USE GRAPHS when:                                    │   │
│  │  • Many small kernels (10+ at <0.1ms each)              │   │
│  │  • Repeated execution (100+ times)                      │   │
│  │  • Fixed topology with variable data                    │   │
│  │  • Training loops, inference pipelines                  │   │
│  │                                                         │   │
│  │  ❌ AVOID GRAPHS when:                                  │   │
│  │  • Dynamic control flow (if/else per iteration)        │   │
│  │  • One-shot execution (setup > savings)                 │   │
│  │  • Large kernels (launch overhead negligible)           │   │
│  │  • Grid/block dimensions change per iteration           │   │
│  │                                                         │   │
│  └─────────────────────────────────────────────────────────┘   │
│                                                                 │
│  BREAKEVEN ANALYSIS:                                            │
│  ┌─────────────────────────────────────────────────────────┐   │
│  │                                                         │   │
│  │  Setup Cost:    Capture (~100μs) + Instantiate (~50μs)  │   │
│  │  Per-Launch:    Regular: ~10μs × N kernels              │   │
│  │                 Graph:   ~5μs total                     │   │
│  │                                                         │   │
│  │  Example (10 kernels):                                  │   │
│  │  • Regular: 10 × 10μs = 100μs per launch                │   │
│  │  • Graph: 150μs setup + 5μs per launch                  │   │
│  │  • Breakeven: 150 ÷ (100-5) ≈ 2 iterations!            │   │
│  │                                                         │   │
│  │  After 1000 iterations:                                 │   │
│  │  • Regular: 1000 × 100μs = 100,000μs                    │   │
│  │  • Graph:   150μs + 1000 × 5μs = 5,150μs                │   │
│  │  • Speedup: ~20x!                                       │   │
│  │                                                         │   │
│  └─────────────────────────────────────────────────────────┘   │
│                                                                 │
│  STRUCTURE OPTIMIZATION:                                        │
│  ┌─────────────────────────────────────────────────────────┐   │
│  │  Before (Deep):        After (Wide):                    │   │
│  │  K1→K2→K3→K4           K1→[K2,K3]→K4                    │   │
│  │  Time: 4 steps         Time: 3 steps (parallel!)        │   │
│  │                                                         │   │
│  │  Optimization Tips:                                     │   │
│  │  • Minimize graph depth (critical path)                 │   │
│  │  • Maximize width (parallel kernels)                    │   │
│  │  • Fuse tiny kernels into larger ones                   │   │
│  │  • Use updates, never rebuild unnecessarily             │   │
│  │                                                         │   │
│  └─────────────────────────────────────────────────────────┘   │
│                                                                 │
│  💡 Key Insight: Graphs are an optimization tool—measure first,│
│     then apply. The best optimization is knowing when NOT to!  │
│                                                                 │
└─────────────────────────────────────────────────────────────────┘
```

---

## Part 1: When to Use Graphs

### Decision Matrix

```
╔══════════════════════════════════════════════════════════╗
║               WHEN TO USE CUDA GRAPHS                    ║
╠══════════════════════════════════════════════════════════╣
║                                                          ║
║  ✅ USE GRAPHS when:                                     ║
║  • Same sequence repeated many times (1000+)             ║
║  • Many small kernels (<0.1ms each)                      ║
║  • Fixed workflow with variable data                     ║
║  • Launch overhead is significant portion of time        ║
║  • Inference in production (fixed model)                 ║
║                                                          ║
║  ❌ AVOID GRAPHS when:                                   ║
║  • Dynamic control flow (if/else, variable loops)        ║
║  • One-shot execution                                    ║
║  • Large kernels where launch overhead negligible        ║
║  • Frequently changing graph topology                    ║
║  • Grid/block dimensions change per iteration            ║
║                                                          ║
╚══════════════════════════════════════════════════════════╝
```

### Breakeven Analysis

```cpp
// When does a graph pay off?

// Costs:
// - Capture:     ~100-500μs
// - Instantiate: ~10-50μs
// - Launch:      ~1-5μs (vs ~5-15μs regular)

// Breakeven example:
// Regular: 10 kernels × 10μs launch = 100μs per iteration
// Graph:   Setup 150μs + 5μs per iteration
// Breakeven at 150μs / (100 - 5)μs ≈ 2 iterations

// After 100 iterations:
// Regular: 100 × 100μs = 10,000μs
// Graph:   150μs + 100 × 5μs = 650μs
// → 15x improvement!
```

---

## Part 2: Graph Structure Optimization

### 🔷 CUDA C++ Implementation (Primary)

This example demonstrates techniques for optimizing graph structure including minimizing depth and fusing operations.

In [ ]:
%%writefile graph_optimization.cu
// graph_optimization.cu - Graph structure optimization
#include <stdio.h>
#include <cuda_runtime.h>

// Technique 1: Minimize graph depth
// ─────────────────────────────────
// BAD: Sequential chain
//   A → B → C → D → E → F
//   Depth = 6, minimal parallelism

// GOOD: Parallel branches
//   A → B → C
//             \→ F
//   D → E →  /
//   Depth = 4, more parallelism

__global__ void kernel(float* d, int n) {
    int tid = blockIdx.x * blockDim.x + threadIdx.x;
    if (tid < n) d[tid] *= 2.0f;
}

// Technique 2: Batch similar operations
// ─────────────────────────────────────
__global__ void fusedKernel(float* a, float* b, float* c, int n) {
    int tid = blockIdx.x * blockDim.x + threadIdx.x;
    if (tid < n) {
        // Multiple operations in one kernel
        float x = a[tid];
        x = x * 2.0f;  // Was kernel 1
        x = x + 1.0f;  // Was kernel 2
        x = sqrtf(x);  // Was kernel 3
        c[tid] = x;
    }
}

int main() {
    const int N = 1 << 20;
    float *d_a, *d_b;
    cudaMalloc(&d_a, N * sizeof(float));
    cudaMalloc(&d_b, N * sizeof(float));
    
    cudaStream_t stream;
    cudaStreamCreate(&stream);
    
    // ============================================
    // Capture Optimized Graph
    // ============================================
    cudaGraph_t graph;
    cudaStreamBeginCapture(stream, cudaStreamCaptureModeGlobal);
    
    // Parallel memcpys (device-to-device)
    cudaMemcpyAsync(d_b, d_a, N/2 * sizeof(float), 
                    cudaMemcpyDeviceToDevice, stream);
    
    // Fused kernel instead of multiple small ones
    fusedKernel<<<256, 256, 0, stream>>>(d_a, d_a, d_b, N);
    
    cudaStreamEndCapture(stream, &graph);
    
    // ============================================
    // Verify Graph Structure
    // ============================================
    size_t numNodes;
    cudaGraphGetNodes(graph, NULL, &numNodes);
    printf("Optimized graph has %zu nodes\n", numNodes);
    
    cudaGraphExec_t graphExec;
    cudaGraphInstantiate(&graphExec, graph, NULL, NULL, 0);
    
    // Warmup
    cudaGraphLaunch(graphExec, stream);
    cudaStreamSynchronize(stream);
    
    // Benchmark
    cudaEvent_t start, stop;
    cudaEventCreate(&start);
    cudaEventCreate(&stop);
    
    const int ITERATIONS = 1000;
    cudaEventRecord(start, stream);
    
    for (int i = 0; i < ITERATIONS; i++) {
        cudaGraphLaunch(graphExec, stream);
    }
    
    cudaEventRecord(stop, stream);
    cudaStreamSynchronize(stream);
    
    float ms;
    cudaEventElapsedTime(&ms, start, stop);
    printf("Average launch time: %.2f μs\n", (ms * 1000) / ITERATIONS);
    
    // Cleanup
    cudaGraphExecDestroy(graphExec);
    cudaGraphDestroy(graph);
    cudaStreamDestroy(stream);
    cudaFree(d_a);
    cudaFree(d_b);
    
    return 0;
}

---

## Part 3: Graphs with Streams

### 🔷 CUDA C++ Implementation (Primary)

This example demonstrates running multiple graphs concurrently on different streams for maximum GPU utilization.

In [ ]:
%%writefile concurrent_graphs.cu
// concurrent_graphs.cu - Running multiple graphs concurrently
#include <stdio.h>
#include <cuda_runtime.h>

__global__ void process(float* data, int n) {
    int tid = blockIdx.x * blockDim.x + threadIdx.x;
    if (tid < n) data[tid] = sqrtf(data[tid] + 1.0f);
}

int main() {
    const int N = 1 << 18;
    const int NUM_GRAPHS = 4;
    
    float* d_data[NUM_GRAPHS];
    cudaStream_t streams[NUM_GRAPHS];
    cudaGraphExec_t graphExecs[NUM_GRAPHS];
    
    // ============================================
    // Create Multiple Graphs
    // ============================================
    for (int i = 0; i < NUM_GRAPHS; i++) {
        cudaMalloc(&d_data[i], N * sizeof(float));
        cudaStreamCreate(&streams[i]);
        
        // Capture graph for this stream
        cudaGraph_t graph;
        cudaStreamBeginCapture(streams[i], cudaStreamCaptureModeGlobal);
        
        process<<<256, 256, 0, streams[i]>>>(d_data[i], N);
        
        cudaStreamEndCapture(streams[i], &graph);
        cudaGraphInstantiate(&graphExecs[i], graph, NULL, NULL, 0);
        cudaGraphDestroy(graph);  // Can destroy after instantiate
    }
    
    // ============================================
    // Launch All Graphs Concurrently
    // ============================================
    // Different graphs on different streams run in parallel!
    
    cudaEvent_t start, stop;
    cudaEventCreate(&start);
    cudaEventCreate(&stop);
    
    cudaEventRecord(start, 0);
    
    for (int iter = 0; iter < 100; iter++) {
        for (int i = 0; i < NUM_GRAPHS; i++) {
            // Each graph on its own stream
            cudaGraphLaunch(graphExecs[i], streams[i]);
        }
    }
    
    for (int i = 0; i < NUM_GRAPHS; i++) {
        cudaStreamSynchronize(streams[i]);
    }
    
    cudaEventRecord(stop, 0);
    cudaEventSynchronize(stop);
    
    float ms;
    cudaEventElapsedTime(&ms, start, stop);
    printf("%d concurrent graphs, 100 iterations: %.2f ms\n", NUM_GRAPHS, ms);
    printf("Average per iteration: %.2f μs\n", (ms * 1000) / 100);
    
    // Cleanup
    for (int i = 0; i < NUM_GRAPHS; i++) {
        cudaGraphExecDestroy(graphExecs[i]);
        cudaStreamDestroy(streams[i]);
        cudaFree(d_data[i]);
    }
    
    return 0;
}

In [ ]:
!nvcc -arch=sm_75 -o concurrent_graphs concurrent_graphs.cu && ./concurrent_graphs

---

## Part 4: Real-World Pattern - Inference Pipeline

### 🔷 CUDA C++ Inference Graph (Primary)

This example demonstrates a practical neural network inference pipeline using CUDA Graphs. The graph captures multiple layer operations (Linear + ReLU activations + Softmax) and benchmarks the throughput achievable with graph-based execution.

In [ ]:
%%writefile inference_graph.cu
// inference_graph.cu - Neural network inference pattern
#include <stdio.h>
#include <cuda_runtime.h>
#include <stdlib.h>

// Simplified layer kernels
__global__ void linearLayer(float* out, const float* in, const float* W, 
                            const float* b, int out_dim, int in_dim) {
    int tid = blockIdx.x * blockDim.x + threadIdx.x;
    if (tid < out_dim) {
        float sum = b[tid];
        for (int i = 0; i < in_dim; i++) {
            sum += W[tid * in_dim + i] * in[i];
        }
        out[tid] = sum;
    }
}

__global__ void relu(float* data, int n) {
    int tid = blockIdx.x * blockDim.x + threadIdx.x;
    if (tid < n) data[tid] = fmaxf(0.0f, data[tid]);
}

__global__ void softmaxKernel(float* out, const float* in, int n) {
    __shared__ float maxVal;
    __shared__ float sum;
    
    int tid = threadIdx.x;
    
    // Find max (simplified for small n)
    if (tid == 0) {
        maxVal = in[0];
        for (int i = 1; i < n; i++) {
            if (in[i] > maxVal) maxVal = in[i];
        }
        sum = 0.0f;
    }
    __syncthreads();
    
    // Compute exp sum
    if (tid < n) {
        float val = expf(in[tid] - maxVal);
        atomicAdd(&sum, val);
    }
    __syncthreads();
    
    // Normalize
    if (tid < n) out[tid] = expf(in[tid] - maxVal) / sum;
}

int main() {
    const int INPUT_DIM = 784;
    const int HIDDEN_DIM = 256;
    const int OUTPUT_DIM = 10;
    
    // Allocate device memory
    float *d_input, *d_hidden1, *d_hidden2, *d_output;
    float *d_W1, *d_b1, *d_W2, *d_b2, *d_W3, *d_b3;
    
    cudaMalloc(&d_input, INPUT_DIM * sizeof(float));
    cudaMalloc(&d_hidden1, HIDDEN_DIM * sizeof(float));
    cudaMalloc(&d_hidden2, HIDDEN_DIM * sizeof(float));
    cudaMalloc(&d_output, OUTPUT_DIM * sizeof(float));
    
    // Weights (simplified - just allocate, don't init)
    cudaMalloc(&d_W1, HIDDEN_DIM * INPUT_DIM * sizeof(float));
    cudaMalloc(&d_b1, HIDDEN_DIM * sizeof(float));
    cudaMalloc(&d_W2, HIDDEN_DIM * HIDDEN_DIM * sizeof(float));
    cudaMalloc(&d_b2, HIDDEN_DIM * sizeof(float));
    cudaMalloc(&d_W3, OUTPUT_DIM * HIDDEN_DIM * sizeof(float));
    cudaMalloc(&d_b3, OUTPUT_DIM * sizeof(float));
    
    cudaStream_t stream;
    cudaStreamCreate(&stream);
    
    // Build inference graph
    cudaGraph_t graph;
    cudaStreamBeginCapture(stream, cudaStreamCaptureModeGlobal);
    
    // Layer 1: Linear + ReLU
    linearLayer<<<1, 256, 0, stream>>>(d_hidden1, d_input, d_W1, d_b1,
                                        HIDDEN_DIM, INPUT_DIM);
    relu<<<1, 256, 0, stream>>>(d_hidden1, HIDDEN_DIM);
    
    // Layer 2: Linear + ReLU
    linearLayer<<<1, 256, 0, stream>>>(d_hidden2, d_hidden1, d_W2, d_b2,
                                        HIDDEN_DIM, HIDDEN_DIM);
    relu<<<1, 256, 0, stream>>>(d_hidden2, HIDDEN_DIM);
    
    // Layer 3: Linear + Softmax
    linearLayer<<<1, 256, 0, stream>>>(d_output, d_hidden2, d_W3, d_b3,
                                        OUTPUT_DIM, HIDDEN_DIM);
    softmaxKernel<<<1, 32, 0, stream>>>(d_output, d_output, OUTPUT_DIM);
    
    cudaStreamEndCapture(stream, &graph);
    
    cudaGraphExec_t graphExec;
    cudaGraphInstantiate(&graphExec, graph, NULL, NULL, 0);
    
    size_t numNodes;
    cudaGraphGetNodes(graph, NULL, &numNodes);
    printf("Inference graph has %zu nodes (6 kernels expected)\n", numNodes);
    
    cudaGraphDestroy(graph);  // Template no longer needed
    
    // Benchmark inference
    cudaEvent_t start, stop;
    cudaEventCreate(&start);
    cudaEventCreate(&stop);
    
    const int ITERATIONS = 10000;
    
    // Warmup
    for (int i = 0; i < 10; i++) {
        cudaGraphLaunch(graphExec, stream);
    }
    cudaStreamSynchronize(stream);
    
    // Benchmark
    cudaEventRecord(start, stream);
    for (int i = 0; i < ITERATIONS; i++) {
        cudaGraphLaunch(graphExec, stream);
    }
    cudaEventRecord(stop, stream);
    cudaStreamSynchronize(stream);
    
    float ms;
    cudaEventElapsedTime(&ms, start, stop);
    printf("Inference: %d iterations in %.2f ms\n", ITERATIONS, ms);
    printf("Per inference: %.2f μs\n", (ms * 1000) / ITERATIONS);
    printf("Throughput: %.0f inferences/sec\n", ITERATIONS / (ms / 1000));
    
    // Cleanup
    cudaGraphExecDestroy(graphExec);
    cudaStreamDestroy(stream);
    cudaFree(d_input);
    cudaFree(d_hidden1);
    cudaFree(d_hidden2);
    cudaFree(d_output);
    cudaFree(d_W1); cudaFree(d_b1);
    cudaFree(d_W2); cudaFree(d_b2);
    cudaFree(d_W3); cudaFree(d_b3);
    
    return 0;
}

In [ ]:
!nvcc -arch=sm_75 -o inference_graph inference_graph.cu && ./inference_graph

---

## Part 5: Best Practices Summary

### Graph Lifecycle

```cpp
// BEST PRACTICE: Proper lifecycle management

// 1. Create/capture graph
cudaGraph_t graph;
// ... capture or build ...

// 2. Instantiate once
cudaGraphExec_t graphExec;
cudaGraphInstantiate(&graphExec, graph, NULL, NULL, 0);

// 3. Destroy template (optional - saves memory)
cudaGraphDestroy(graph);

// 4. Launch many times
for (int i = 0; i < 1000000; i++) {
    // Update if needed
    if (parametersChanged) {
        cudaGraphExecKernelNodeSetParams(...);
    }
    cudaGraphLaunch(graphExec, stream);
}

// 5. Cleanup at end
cudaGraphExecDestroy(graphExec);
```

### Common Mistakes

```
❌ MISTAKE 1: Rebuilding graphs every iteration
   → Build once, update parameters

❌ MISTAKE 2: Using graphs for single-shot work
   → Use regular launches for one-off tasks

❌ MISTAKE 3: Capturing allocations in graph
   → Allocate outside, only use inside

❌ MISTAKE 4: Forgetting to sync before reading results
   → Always cudaStreamSynchronize before host access

❌ MISTAKE 5: Using host-side conditionals in capture
   → All conditionals evaluated at capture time!
```

---

## 🎯 Exercises

### 🔷 CUDA C++ Exercises (Primary)

In [ ]:
%%writefile graph_optimization_exercises.cu
/*
 * CUDA Graph Optimization Exercises
 * Exercise 1: Breakeven Analysis - When graphs beat regular launches
 * Exercise 2: Multi-Stream Graphs - Concurrent execution within graphs
 * Exercise 3: Complete Inference Pipeline - Full pipeline with memcpy nodes
 */

#include <cuda_runtime.h>
#include <stdio.h>
#include <chrono>

#define CHECK_CUDA(call) \
    do { \
        cudaError_t err = call; \
        if (err != cudaSuccess) { \
            printf("CUDA error at %s:%d: %s\n", __FILE__, __LINE__, \
                   cudaGetErrorString(err)); \
            exit(1); \
        } \
    } while(0)

// Simple kernel for benchmarking
__global__ void simpleKernel(float* data, int n) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    if (idx < n) {
        data[idx] = data[idx] * 2.0f + 1.0f;
    }
}

// Simulated layer kernels for inference pipeline
__global__ void layer1Kernel(float* input, float* output, int n) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    if (idx < n) {
        output[idx] = tanhf(input[idx]);
    }
}

__global__ void layer2Kernel(float* input, float* output, int n) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    if (idx < n) {
        output[idx] = input[idx] * 0.5f + 0.5f;  // Sigmoid approx
    }
}

__global__ void layer3Kernel(float* input, float* output, int n) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    if (idx < n) {
        output[idx] = fmaxf(0.0f, input[idx]);  // ReLU
    }
}

// =============================================================
// Exercise 1: Breakeven Analysis
// Measure at what iteration count graphs become faster
// =============================================================
void exercise1_breakeven_analysis() {
    printf("\n=== Exercise 1: Breakeven Analysis ===\n");
    
    const int N = 1024;  // Small N to emphasize launch overhead
    const int BLOCK_SIZE = 256;
    const int GRID_SIZE = (N + BLOCK_SIZE - 1) / BLOCK_SIZE;
    
    float* d_data;
    CHECK_CUDA(cudaMalloc(&d_data, N * sizeof(float)));
    CHECK_CUDA(cudaMemset(d_data, 0, N * sizeof(float)));
    
    cudaStream_t stream;
    CHECK_CUDA(cudaStreamCreate(&stream));
    
    // Create graph
    cudaGraph_t graph;
    cudaGraphExec_t graphExec;
    
    CHECK_CUDA(cudaStreamBeginCapture(stream, cudaStreamCaptureModeGlobal));
    simpleKernel<<<GRID_SIZE, BLOCK_SIZE, 0, stream>>>(d_data, N);
    CHECK_CUDA(cudaStreamEndCapture(stream, &graph));
    CHECK_CUDA(cudaGraphInstantiate(&graphExec, graph, NULL, NULL, 0));
    
    // Test different iteration counts
    int testCounts[] = {1, 5, 10, 25, 50, 100, 200, 500, 1000};
    int numTests = sizeof(testCounts) / sizeof(testCounts[0]);
    
    printf("\n%-12s %-15s %-15s %-10s\n", "Iterations", "Regular (µs)", "Graph (µs)", "Winner");
    printf("%-12s %-15s %-15s %-10s\n", "----------", "-----------", "---------", "------");
    
    int breakeven = -1;
    
    for (int t = 0; t < numTests; t++) {
        int iters = testCounts[t];
        
        // Warm up
        for (int i = 0; i < 10; i++) {
            simpleKernel<<<GRID_SIZE, BLOCK_SIZE, 0, stream>>>(d_data, N);
            CHECK_CUDA(cudaGraphLaunch(graphExec, stream));
        }
        CHECK_CUDA(cudaStreamSynchronize(stream));
        
        // Time regular launches
        auto start = std::chrono::high_resolution_clock::now();
        for (int i = 0; i < iters; i++) {
            simpleKernel<<<GRID_SIZE, BLOCK_SIZE, 0, stream>>>(d_data, N);
        }
        CHECK_CUDA(cudaStreamSynchronize(stream));
        auto end = std::chrono::high_resolution_clock::now();
        double regularTime = std::chrono::duration<double, std::micro>(end - start).count();
        
        // Time graph launches
        start = std::chrono::high_resolution_clock::now();
        for (int i = 0; i < iters; i++) {
            CHECK_CUDA(cudaGraphLaunch(graphExec, stream));
        }
        CHECK_CUDA(cudaStreamSynchronize(stream));
        end = std::chrono::high_resolution_clock::now();
        double graphTime = std::chrono::duration<double, std::micro>(end - start).count();
        
        const char* winner = graphTime < regularTime ? "Graph" : "Regular";
        printf("%-12d %-15.2f %-15.2f %-10s\n", iters, regularTime, graphTime, winner);
        
        if (breakeven == -1 && graphTime < regularTime) {
            breakeven = iters;
        }
    }
    
    if (breakeven != -1) {
        printf("\nBreakeven point: Around %d iterations\n", breakeven);
    } else {
        printf("\nGraph not faster in tested range\n");
    }
    
    CHECK_CUDA(cudaGraphExecDestroy(graphExec));
    CHECK_CUDA(cudaGraphDestroy(graph));
    CHECK_CUDA(cudaStreamDestroy(stream));
    CHECK_CUDA(cudaFree(d_data));
}

// =============================================================
// Exercise 2: Multi-Stream Graphs
// Create a graph with parallel branches using stream forking
// =============================================================
void exercise2_multi_stream_graphs() {
    printf("\n=== Exercise 2: Multi-Stream Graphs ===\n");
    
    const int N = 1024 * 256;
    const int BLOCK_SIZE = 256;
    const int GRID_SIZE = (N + BLOCK_SIZE - 1) / BLOCK_SIZE;
    
    float *d_A, *d_B, *d_C;
    CHECK_CUDA(cudaMalloc(&d_A, N * sizeof(float)));
    CHECK_CUDA(cudaMalloc(&d_B, N * sizeof(float)));
    CHECK_CUDA(cudaMalloc(&d_C, N * sizeof(float)));
    
    cudaStream_t mainStream, forkStreamA, forkStreamB;
    CHECK_CUDA(cudaStreamCreate(&mainStream));
    CHECK_CUDA(cudaStreamCreate(&forkStreamA));
    CHECK_CUDA(cudaStreamCreate(&forkStreamB));
    
    // Create graph with parallel branches (fork-join pattern)
    cudaGraph_t graph;
    cudaGraphExec_t graphExec;
    
    CHECK_CUDA(cudaStreamBeginCapture(mainStream, cudaStreamCaptureModeGlobal));
    
    // Fork into parallel streams
    cudaEvent_t forkEvent, joinEventA, joinEventB;
    CHECK_CUDA(cudaEventCreate(&forkEvent));
    CHECK_CUDA(cudaEventCreate(&joinEventA));
    CHECK_CUDA(cudaEventCreate(&joinEventB));
    
    // Record fork point
    CHECK_CUDA(cudaEventRecord(forkEvent, mainStream));
    CHECK_CUDA(cudaStreamWaitEvent(forkStreamA, forkEvent, 0));
    CHECK_CUDA(cudaStreamWaitEvent(forkStreamB, forkEvent, 0));
    
    // Parallel work on branches
    simpleKernel<<<GRID_SIZE, BLOCK_SIZE, 0, forkStreamA>>>(d_A, N);
    simpleKernel<<<GRID_SIZE, BLOCK_SIZE, 0, forkStreamA>>>(d_A, N);
    
    simpleKernel<<<GRID_SIZE, BLOCK_SIZE, 0, forkStreamB>>>(d_B, N);
    simpleKernel<<<GRID_SIZE, BLOCK_SIZE, 0, forkStreamB>>>(d_B, N);
    
    // Join back
    CHECK_CUDA(cudaEventRecord(joinEventA, forkStreamA));
    CHECK_CUDA(cudaEventRecord(joinEventB, forkStreamB));
    CHECK_CUDA(cudaStreamWaitEvent(mainStream, joinEventA, 0));
    CHECK_CUDA(cudaStreamWaitEvent(mainStream, joinEventB, 0));
    
    // Final kernel after join
    simpleKernel<<<GRID_SIZE, BLOCK_SIZE, 0, mainStream>>>(d_C, N);
    
    CHECK_CUDA(cudaStreamEndCapture(mainStream, &graph));
    CHECK_CUDA(cudaGraphInstantiate(&graphExec, graph, NULL, NULL, 0));
    
    // Analyze graph structure
    size_t numNodes, numEdges;
    CHECK_CUDA(cudaGraphGetNodes(graph, nullptr, &numNodes));
    CHECK_CUDA(cudaGraphGetEdges(graph, nullptr, nullptr, &numEdges));
    
    printf("Graph structure:\n");
    printf("  Nodes: %zu\n", numNodes);
    printf("  Edges: %zu\n", numEdges);
    printf("  Pattern: Fork-Join (2 parallel branches)\n");
    
    // Execute and time
    auto start = std::chrono::high_resolution_clock::now();
    for (int i = 0; i < 100; i++) {
        CHECK_CUDA(cudaGraphLaunch(graphExec, mainStream));
    }
    CHECK_CUDA(cudaStreamSynchronize(mainStream));
    auto end = std::chrono::high_resolution_clock::now();
    double graphTime = std::chrono::duration<double, std::micro>(end - start).count();
    
    printf("  100 launches: %.2f µs (%.2f µs/launch)\n", graphTime, graphTime / 100);
    
    // Cleanup
    CHECK_CUDA(cudaEventDestroy(forkEvent));
    CHECK_CUDA(cudaEventDestroy(joinEventA));
    CHECK_CUDA(cudaEventDestroy(joinEventB));
    CHECK_CUDA(cudaGraphExecDestroy(graphExec));
    CHECK_CUDA(cudaGraphDestroy(graph));
    CHECK_CUDA(cudaStreamDestroy(mainStream));
    CHECK_CUDA(cudaStreamDestroy(forkStreamA));
    CHECK_CUDA(cudaStreamDestroy(forkStreamB));
    CHECK_CUDA(cudaFree(d_A));
    CHECK_CUDA(cudaFree(d_B));
    CHECK_CUDA(cudaFree(d_C));
}

// =============================================================
// Exercise 3: Complete Inference Pipeline
// Full pipeline with memcpy nodes for input/output
// =============================================================
void exercise3_complete_inference_pipeline() {
    printf("\n=== Exercise 3: Complete Inference Pipeline ===\n");
    
    const int N = 1024 * 32;  // Batch size * features
    const int BLOCK_SIZE = 256;
    const int GRID_SIZE = (N + BLOCK_SIZE - 1) / BLOCK_SIZE;
    
    // Host memory (pinned for async transfers)
    float *h_input, *h_output;
    CHECK_CUDA(cudaMallocHost(&h_input, N * sizeof(float)));
    CHECK_CUDA(cudaMallocHost(&h_output, N * sizeof(float)));
    
    // Initialize input
    for (int i = 0; i < N; i++) {
        h_input[i] = (float)(i % 100) / 100.0f - 0.5f;
    }
    
    // Device memory
    float *d_input, *d_layer1, *d_layer2, *d_output;
    CHECK_CUDA(cudaMalloc(&d_input, N * sizeof(float)));
    CHECK_CUDA(cudaMalloc(&d_layer1, N * sizeof(float)));
    CHECK_CUDA(cudaMalloc(&d_layer2, N * sizeof(float)));
    CHECK_CUDA(cudaMalloc(&d_output, N * sizeof(float)));
    
    // Build explicit graph with memcpy + compute nodes
    cudaGraph_t graph;
    cudaGraphExec_t graphExec;
    CHECK_CUDA(cudaGraphCreate(&graph, 0));
    
    // Node 1: Copy input H2D
    cudaGraphNode_t memcpyH2DNode;
    cudaMemcpy3DParms h2dParams = {0};
    h2dParams.srcPtr = make_cudaPitchedPtr(h_input, N * sizeof(float), N, 1);
    h2dParams.dstPtr = make_cudaPitchedPtr(d_input, N * sizeof(float), N, 1);
    h2dParams.extent = make_cudaExtent(N * sizeof(float), 1, 1);
    h2dParams.kind = cudaMemcpyHostToDevice;
    CHECK_CUDA(cudaGraphAddMemcpyNode(&memcpyH2DNode, graph, nullptr, 0, &h2dParams));
    
    // Node 2: Layer 1 kernel
    cudaGraphNode_t layer1Node;
    void* layer1Args[] = { &d_input, &d_layer1, (void*)&N };
    cudaKernelNodeParams layer1Params = {0};
    layer1Params.func = (void*)layer1Kernel;
    layer1Params.gridDim = dim3(GRID_SIZE);
    layer1Params.blockDim = dim3(BLOCK_SIZE);
    layer1Params.kernelParams = layer1Args;
    CHECK_CUDA(cudaGraphAddKernelNode(&layer1Node, graph, &memcpyH2DNode, 1, &layer1Params));
    
    // Node 3: Layer 2 kernel
    cudaGraphNode_t layer2Node;
    void* layer2Args[] = { &d_layer1, &d_layer2, (void*)&N };
    cudaKernelNodeParams layer2Params = {0};
    layer2Params.func = (void*)layer2Kernel;
    layer2Params.gridDim = dim3(GRID_SIZE);
    layer2Params.blockDim = dim3(BLOCK_SIZE);
    layer2Params.kernelParams = layer2Args;
    CHECK_CUDA(cudaGraphAddKernelNode(&layer2Node, graph, &layer1Node, 1, &layer2Params));
    
    // Node 4: Layer 3 kernel
    cudaGraphNode_t layer3Node;
    void* layer3Args[] = { &d_layer2, &d_output, (void*)&N };
    cudaKernelNodeParams layer3Params = {0};
    layer3Params.func = (void*)layer3Kernel;
    layer3Params.gridDim = dim3(GRID_SIZE);
    layer3Params.blockDim = dim3(BLOCK_SIZE);
    layer3Params.kernelParams = layer3Args;
    CHECK_CUDA(cudaGraphAddKernelNode(&layer3Node, graph, &layer2Node, 1, &layer3Params));
    
    // Node 5: Copy output D2H
    cudaGraphNode_t memcpyD2HNode;
    cudaMemcpy3DParms d2hParams = {0};
    d2hParams.srcPtr = make_cudaPitchedPtr(d_output, N * sizeof(float), N, 1);
    d2hParams.dstPtr = make_cudaPitchedPtr(h_output, N * sizeof(float), N, 1);
    d2hParams.extent = make_cudaExtent(N * sizeof(float), 1, 1);
    d2hParams.kind = cudaMemcpyDeviceToHost;
    CHECK_CUDA(cudaGraphAddMemcpyNode(&memcpyD2HNode, graph, &layer3Node, 1, &d2hParams));
    
    // Instantiate
    CHECK_CUDA(cudaGraphInstantiate(&graphExec, graph, NULL, NULL, 0));
    
    // Print graph info
    size_t numNodes;
    CHECK_CUDA(cudaGraphGetNodes(graph, nullptr, &numNodes));
    printf("Inference pipeline graph:\n");
    printf("  Total nodes: %zu\n", numNodes);
    printf("  Structure: H2D -> Layer1 -> Layer2 -> Layer3 -> D2H\n");
    
    // Execute pipeline
    cudaStream_t stream;
    CHECK_CUDA(cudaStreamCreate(&stream));
    
    auto start = std::chrono::high_resolution_clock::now();
    const int NUM_INFERENCES = 100;
    for (int i = 0; i < NUM_INFERENCES; i++) {
        CHECK_CUDA(cudaGraphLaunch(graphExec, stream));
    }
    CHECK_CUDA(cudaStreamSynchronize(stream));
    auto end = std::chrono::high_resolution_clock::now();
    
    double totalTime = std::chrono::duration<double, std::milli>(end - start).count();
    printf("\nPerformance:\n");
    printf("  %d inferences: %.2f ms\n", NUM_INFERENCES, totalTime);
    printf("  Per inference: %.3f ms\n", totalTime / NUM_INFERENCES);
    printf("  Throughput: %.1f inferences/sec\n", NUM_INFERENCES / (totalTime / 1000.0));
    
    // Verify output (sample)
    printf("\nSample output (first 5 elements):\n");
    for (int i = 0; i < 5; i++) {
        printf("  output[%d] = %.4f\n", i, h_output[i]);
    }
    
    // Cleanup
    CHECK_CUDA(cudaGraphExecDestroy(graphExec));
    CHECK_CUDA(cudaGraphDestroy(graph));
    CHECK_CUDA(cudaStreamDestroy(stream));
    CHECK_CUDA(cudaFreeHost(h_input));
    CHECK_CUDA(cudaFreeHost(h_output));
    CHECK_CUDA(cudaFree(d_input));
    CHECK_CUDA(cudaFree(d_layer1));
    CHECK_CUDA(cudaFree(d_layer2));
    CHECK_CUDA(cudaFree(d_output));
}

int main() {
    printf("CUDA Graph Optimization Exercises\n");
    printf("==================================\n");
    
    int device;
    cudaDeviceProp prop;
    CHECK_CUDA(cudaGetDevice(&device));
    CHECK_CUDA(cudaGetDeviceProperties(&prop, device));
    printf("Device: %s\n", prop.name);
    
    exercise1_breakeven_analysis();
    exercise2_multi_stream_graphs();
    exercise3_complete_inference_pipeline();
    
    printf("\n✓ All exercises completed!\n");
    return 0;
}

In [ ]:
!nvcc -arch=sm_75 -o graph_optimization_exercises graph_optimization_exercises.cu && ./graph_optimization_exercises

### 🔶 Python/Numba Exercises (Optional)

### Exercise 1: Breakeven Analysis
Measure at what iteration count graphs become faster than regular launches.

### Exercise 2: Multi-Stream Graphs
Create a graph that uses multiple internal streams.

### Exercise 3: Complete Inference Pipeline
Extend the inference example with memcpy nodes in the graph.

---

## Key Takeaways

```
┌─────────────────────────────────────────────────────────┐
│       GRAPH OPTIMIZATION: FINDING THE FASTEST RECIPE    │
├─────────────────────────────────────────────────────────┤
│                                                         │
│  🎯 Pattern: Measure, Then Optimize                     │
│  • Not all code benefits from graphs                    │
│  • The best optimization is knowing when NOT to use it! │
│  • Always calculate breakeven before adding complexity  │
│                                                         │
│  When to Use:                                           │
│  • Many small kernels (launch-bound, not compute-bound) │
│  • Repeated execution (100+ times)                      │
│  • Fixed topology with variable data                    │
│                                                         │
│  Optimization Tips:                                     │
│  • Minimize graph depth (shorter critical path)         │
│  • Maximize parallelism (fork-join patterns)            │
│  • Fuse small kernels when possible                     │
│  • Use updates, never rebuild unnecessarily             │
│                                                         │
│  Lifecycle Best Practices:                              │
│  • Capture/build → Instantiate → Launch(N) → Destroy    │
│  • Can destroy template graph after instantiate         │
│  • Keep exec alive for entire repetition sequence       │
│                                                         │
│  Concurrency:                                           │
│  • Different graphs on different streams = parallel     │
│  • Same graphExec = serialized                          │
│  • Multiple graphExec from same template = parallel OK  │
│                                                         │
│  ⚠️ Week 10 Summary:                                    │
│  • Day 1: Capture (record cooking → recipe)             │
│  • Day 2: Explicit (write recipes from scratch)         │
│  • Day 3: Updates (change ingredients, keep recipe)     │
│  • Day 4: Optimize (chef's efficiency analysis)         │
│                                                         │
└─────────────────────────────────────────────────────────┘
```

## 🎉 Week 10 Complete: CUDA Graphs Mastery!

You've learned to think like a **head chef** managing a GPU kitchen:
- **Recipe Cards** (graphs) eliminate per-instruction overhead
- **Capture** records existing cooking into recipes
- **Explicit Build** designs custom parallel workflows
- **Updates** change ingredients without rewriting recipes
- **Optimization** knows when recipes help vs. hurt

## What's Next?

With CUDA Graphs, you can now **eliminate CPU-GPU coordination overhead**. But what about coordination **within the GPU itself**? Next week explores **Cooperative Groups & Dynamic Parallelism**—letting GPU threads coordinate and launch their own work!

**→ Week 11: Cooperative Groups & Dynamic Parallelism** (Threads that think for themselves)